In [137]:
%pip install langchain_community langchain_ollama transformers langchain_huggingface --quiet 

Note: you may need to restart the kernel to use updated packages.


In [155]:
from langchain_ollama.llms import OllamaLLM
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.docstore.document import Document
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate
import pandas as pd
import os

In [156]:
data = "D:\Code\local-agent-notebooks\datasets\cafef-news-09-07-2025.csv"
df = pd.read_csv(data)
df.head()

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Admin\AppData\Local\Temp\ipykernel_27248\1940478403.py:1: SyntaxWarning: invalid escape sequence '\C'
  data = "D:\Code\local-agent-notebooks\datasets\cafef-news-09-07-2025.csv"


,Unnamed: 0,url,title,published,summary,body
0,59,https://cafef.vn/dn-lien-quan-truong-my-lan-na...,DN liên quan Trương Mỹ Lan nắm trong tay loạt ...,2025-09-07 11:51:00,Đơn vị kiểm toán (Công ty A&C) đã từ chối đưa ...,MỚI NHẤT!\n\n07-09-2025 - 11:51 AM | Doanh ngh...
1,60,https://cafef.vn/tin-vui-cho-evn-1882509071135...,Tin vui cho EVN,2025-09-07 11:35:00,Lần gần nhất có thay đổi giá bán lẻ điện bình ...,MỚI NHẤT!\n\n07-09-2025 - 11:35 AM | Doanh ngh...
2,11,https://cafef.vn/gia-nha-tang-cao-chung-cu-so-...,"Giá nhà tăng cao, chung cư sở hữu có thời hạn ...",2025-09-07 11:23:00,"Thời gian gần đây, giá nhà tăng quá cao khiến ...",MỚI NHẤT!\n\n07-09-2025 - 11:23 AM | Bất động ...
3,70,https://cafef.vn/giam-doc-proship-nam-2015-gia...,Giám đốc Proship: Năm 2015 giao hàng nội thành...,2025-09-07 11:23:00,Vị này lý giải chi phí giao hàng hiện đã rẻ hơ...,MỚI NHẤT!\n\n07-09-2025 - 11:23 AM | Doanh ngh...
4,12,https://cafef.vn/can-canh-khu-nha-xuong-700m2-...,Cận cảnh khu nhà xưởng 700m2 cháy rụi sau vài ...,2025-09-07 11:22:00,"Khu nhà xưởng ở phường Việt Hưng, TP Hà Nội có...",MỚI NHẤT!\n\n07-09-2025 - 11:22 AM | Bất động ...


In [157]:
example = texts = df['body'][1]
example

'MỚI NHẤT!\n\n07-09-2025 - 11:35 AM | Doanh nghiệp\n\nBộ Công Thương vừa có tờ trình về dự thảo sửa đổi bổ sung Nghị định 72/2025 về cơ chế, thời gian điều chỉnh giá bán lẻ điện bình quân. Đây là dự thảo liên quan đến việc cho phép Tập đoàn Điện lực Việt Nam (EVN) thu hồi khoản lỗ của các năm trước thông qua việc điều chỉnh giá bán lẻ điện trong các năm tiếp theo.\n\nCụ thể, EVN cho biết lỗ lũy kế năm 2022-2023 khoảng 50.029 tỷ đồng. Còn riêng công ty mẹ EVN, lỗ lũy kế đến hết năm 2024 khoảng 44.792 tỷ đồng.\ufeff\n\nBộ Công thương đưa ra 2 phương án để xin ý kiến về việc thu hồi khoản lỗ.\n\nPhương án 1: Cho phép phân bổ các khoản lỗ phát sinh từ năm 2022 trở đi vào giá điện của các năm sau. Đây là một cơ chế áp dụng lâu dài.\n\nPhương án 2: Chỉ cho phép xử lý các khoản lỗ phát sinh từ năm 2022 đến trước khi Nghị định sửa đổi có hiệu lực. Phương án này chỉ nhằm xử lý vấn đề đã tồn tại.\n\nVề tác động của sửa đổi nghị định, Bộ\xa0Công Thương cho biết tình hình thủy văn 7 tháng đầu năm 

In [159]:
sample2 = '''
Ước tính, số cổ phiếu thưởng giúp cán bộ nhân viên được hưởng lợi khoảng 844 tỷ đồng, còn việc mua cổ phiếu ưu đãi có thể hưởng lợi hơn 385 tỷ đồng.
06-09-2025Một công ty của Novaland huy động 315 tỷ đồng trái phiếu
03-09-2025Kiểm toán lưu ý về khả năng hoạt động liên tục, Novaland bán 7 tài sản thu về...
30-08-2025Hơn 94.309 tỷ đồng trái phiếu đến hạn trong phần còn lại của năm 2025, "đỉnh...
NVL: Công ty cổ phần Tập đoàn Đầu tư Địa ốc No Va
Giá hiện tại
16.7
Thay đổi
  -0.7 (-3.8%)
Cập nhật lúc 11:00 Thứ 2, 08/09/2025, 11:29

Xem hồ sơ doanh nghiệp 
TIN MỚI
Novaland sau lời khẳng định sẽ tái sinh của ông Bùi Thành Nhơn: Loạt dự án tỷ USD được hồi sinh, “núi nợ” hơn 186.000 tỷ đồng vẫn đè nặng
Novaland sau lời khẳng định sẽ tái sinh của ông Bùi Thành Nhơn: Loạt dự án tỷ USD được hồi sinh, “núi nợ” hơn 186.000 tỷ đồng vẫn đè nặng
Rục rịch đưa 
Rục rịch đưa "Gà đẻ trứng vàng" lên sàn, cổ phiếu loạt doanh nghiệp Việt tăng "bốc đầu"
Thu nhập lãnh đạo Vietnam Airlines tăng 25%
Thu nhập lãnh đạo Vietnam Airlines tăng 25%
Công ty Cổ phần Tập đoàn Đầu tư Địa ốc No Va (Novaland, mã CK: NVL) vừa công bố kế hoạch phát hành cổ phiếu thưởng và chào bán cổ phiếu ưu đãi cho người lao động.

Theo đó, Novaland sẽ phát hành gần 48,8 triệu cổ phiếu để thưởng cho nhân viên, chiếm khoảng 2,5% lượng cổ phiếu đang lưu hành. Nguồn vốn được trích từ thặng dư vốn cổ phần trên báo cáo tài chính riêng năm 2024 đã kiểm toán.

Các cổ phiếu thưởng sẽ bị hạn chế chuyển nhượng: đối với nhân viên thông thường, thời gian hạn chế là 1 năm; còn với nhân sự chủ chốt, mức hạn chế sẽ kéo dài trong 3 năm với lộ trình giảm dần (100% năm đầu, 70% năm thứ hai, 40% năm thứ ba) trước khi được tự do giao dịch từ năm thứ tư.

Bên cạnh đó, Tập đoàn sẽ chào bán gần 48,8 triệu cổ phiếu ưu đãi cho người lao động với giá 10.000 đồng/cổ phiếu, thấp hơn gần 60% so với mức giá 17.300 đồng của NVL trên sàn ở phiên cuối tuần vừa qua.

Tổng cộng, khoảng 97,5 triệu cổ phiếu – tương ứng 5% số lượng cổ phiếu đang lưu hành – sẽ được phát hành trong giai đoạn từ 5/9 đến 30/9/2025. Sau khi hoàn tất, vốn điều lệ của Novaland dự kiến tăng lên 20.476 tỷ đồng.

Ước tính, số cổ phiếu thưởng giúp cán bộ nhân viên được hưởng lợi khoảng 844 tỷ đồng, còn việc mua cổ phiếu ưu đãi có thể hưởng lợi hơn 385 tỷ đồng. Tuy vậy, phần lợi nhuận thực tế từ chương trình ưu đãi này vẫn phụ thuộc vào diễn biến giá cổ phiếu NVL trong thời gian khóa lệnh chuyển nhượng.


Trong nửa đầu năm 2025, Novaland ghi nhận tổng doanh thu hợp nhất đạt 3.715 tỷ đồng, tăng 62,5% so với cùng kỳ năm 2024. Trong đó, mảng bán hàng đóng góp gần 3.423 tỷ đồng, tăng tới 81% nhờ bàn giao tại các dự án trọng điểm như NovaWorld Phan Thiết, NovaWorld Hồ Tràm, Aqua City, Sunrise Riverside, Palm City… Ngoài ra, mảng dịch vụ mang về 291 tỷ đồng.

Dù vẫn lỗ sau thuế 666 tỷ đồng do ảnh hưởng từ chênh lệch tỷ giá và các khoản mục khác, mức lỗ này đã giảm mạnh so với khoản lỗ 7.327 tỷ đồng trong cùng kỳ năm trước.

Tính đến cuối quý II/2025, tổng dư nợ vay tài chính của Novaland ở mức 61.831 tỷ đồng. Trong đó, nợ trái phiếu chiếm tỷ trọng lớn, gần 49%, tương đương 30.290 tỷ đồng. ﻿
'''

In [ ]:
os.environ["GOOGLE_API_KEY"] = ""

In [161]:
llm = OllamaLLM(model="granite3.3:8b", temperature=0)
llm_gemini = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

In [162]:
prompt_template = """Write a concise summary of the following news article.:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

In [163]:
docs = [Document(page_content=sample2)]

In [164]:
# Summarization chain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

In [165]:
result = stuff_chain.invoke(docs)

In [166]:
result.keys()

dict_keys(['input_documents', 'output_text'])

In [167]:
print(result['output_text'])

Novaland, a real estate investment group (CK: NVL), has announced plans to issue approximately 48.8 million shares as employee rewards and for sale in an employee stock offering. The reward shares will constitute about 2.5% of the outstanding shares and will be restricted from transfer for one to three years, depending on employee seniority. The offering price is set at 10,000 VND per share, significantly lower than the recent market price of 17,300 VND.

The company aims to issue a total of around 97.5 million shares (5% of outstanding shares) between September 5 and 30, 2025, increasing its authorized capital to approximately 20.476 billion VND. The estimated benefits for employees from reward shares are around 844 billion VND, while potential gains from the employee stock offering could reach 385 billion VND. However, actual profits depend on NVL's share price fluctuations during the restriction period.

In H1 2025, Novaland reported its highest-ever consolidated revenue of 3,715 bi

In [124]:
llm_chain = LLMChain(llm=llm_gemini, prompt=prompt)
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")
result = stuff_chain.invoke(docs)
print(result['output_text'])

Novaland (NVL) plans to issue approximately 97.5 million shares (5% of outstanding) to employees between September 5-30, 2025. This includes nearly 48.8 million bonus shares from capital surplus, with transfer restrictions of 1-3 years, and another 48.8 million preferential shares offered at 10,000 VND each, significantly below the current market price. Employees are estimated to benefit by over 1.2 trillion VND from this program. Financially, Novaland reported 3.7 trillion VND in revenue for H1 2025 (up 62.5% YoY) and reduced its net loss to 666 billion VND, though it still carries 61.8 trillion VND in financial debt.


In [168]:
# Communication Services/Dịch vụ viễn thông
# Consumer Discretionary/Hàng tiêu dùng
# Consumer Staples/Hàng tiêu dùng thiết yếu
# Health Care/Chăm sóc sức khỏe
# Financial/Tài chính
# Materials/Nguyên vật liệu
# Utilities/Dịch vụ tiện ích
# Energy/Năng lượng
# Industrials/Công nghiệp
# Information Technology/Công nghệ thông tin
# Real Estate/Bất động sản

prompt = PromptTemplate(
    input_variables=["news"],
    template="""
    Given this news:
		{news}

		Identify which stock market sectors are affected.
		Valid sectors are limited to::
		utilities, industrials, consumer discretionary, consumer staples, financials, energy, real estate, IT, materials, health care, communication.  

		If the news mentions any stocks, add their ticker codes alongside the relevant sector.

		Output strictly as a list of sector names (and tickers if available).
    """
)

In [169]:
llm2 = OllamaLLM(model="hf.co/unsloth/gemma-3-12b-it-GGUF:Q4_K_M", temperature=0)
chain = LLMChain(llm=llm2, prompt=prompt)

news = result['output_text']
sectors = chain.run(news)

In [171]:
print(sectors)

*   Real Estate (NVL)
*   Financials


In [133]:
chain = LLMChain(llm=llm_gemini, prompt=prompt)

sectors2 = chain.run(news)

In [134]:
print(sectors2)

*   real estate (NVL)
*   financials


In [182]:
from langchain.chains import LLMChain
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

sentences = [news]

inputs = tokenizer(sentences, return_tensors="pt", padding=True)
outputs = finbert(**inputs)[0]
labels = {0:'neutral', 1:'positive',2:'negative'}
outputs

# Example news
# news = [
#     {'text':result['output_text']}
#     # {"ticker": "AAPL", "text": "Apple beats expectations with record iPhone sales."},
#     # {"ticker": "AAPL", "text": "Apple faces antitrust lawsuit over App Store practices."},
#     # {"ticker": "MSFT", "text": "Microsoft announces strong growth in Azure cloud services."},
    
# ]

# # Run chain
# results = []
# for item in news:
#     output = finbert(item["text"])
#     # FinBERT outputs list of dicts: [{'label': 'positive', 'score': 0.9}, ...]
#     scores = {d['label']: d['score'] for d in output[0]}
#     sentiment_score = scores['positive'] - scores['negative']
#     # results.append({"ticker": item["ticker"], "text": item["text"], "sentiment_score": sentiment_score})

# df = pd.DataFrame(results)
# # agg = df.groupby("ticker")["sentiment_score"].mean().reset_index()

# print(df)
# # print(agg)


tensor([[ 1.9468, -1.4083, -4.2046]], grad_fn=<AddmmBackward0>)